<a href="https://colab.research.google.com/github/JoelAdams001/silver-spoon/blob/master/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn import datasets, linear_model
import statsmodels.api as sm


lin_Data = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')
log_Data = pd.read_csv('https://raw.githubusercontent.com/nborwankar/LearnDataScience/master/datasets/loanf.csv')

In [120]:
lin_Data.head()

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
81174,20000,20000.0,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14.0,14272.0,2.0,< 1 year
99592,19200,19200.0,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12.0,11140.0,1.0,2 years
80059,35000,35000.0,21.98%,60 months,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,690-694,14.0,21977.0,1.0,2 years
15825,10000,9975.0,9.99%,36 months,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,695-699,10.0,9346.0,0.0,5 years
33182,12000,12000.0,11.71%,36 months,credit_card,18.78%,NJ,RENT,3195.00,695-699,11.0,14469.0,0.0,9 years


In [0]:
#Some cleaning you provided on collab
lin_Data['Interest.Rate'] = [float(i[:-1]) for i in lin_Data['Interest.Rate']]
lin_Data['Loan.Length']= [float(i.split(" ")[0]) for i in lin_Data['Loan.Length']]
lin_Data['FICO.Range'] = [(float(i.split("-")[0])+float(i.split("-")[1]))/2 for i in lin_Data['FICO.Range']]

In [122]:
lin_Data.head()

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
81174,20000,20000.0,8.90,36.0,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,737.0,14.0,14272.0,2.0,< 1 year
99592,19200,19200.0,12.12,36.0,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,717.0,12.0,11140.0,1.0,2 years
80059,35000,35000.0,21.98,60.0,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,692.0,14.0,21977.0,1.0,2 years
15825,10000,9975.0,9.99,36.0,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,697.0,10.0,9346.0,0.0,5 years
33182,12000,12000.0,11.71,36.0,credit_card,18.78%,NJ,RENT,3195.00,697.0,11.0,14469.0,0.0,9 years


In [123]:
#This cell also is just the data cleaning that was shared on collab

df = lin_Data[['Interest.Rate', 'Loan.Length', 'FICO.Range']]
intrate = df['Interest.Rate']
loanamt = df['Loan.Length']
fico = df['FICO.Range']

# reshape the data from a pandas Series to columns 
# the dependent variable
y = np.array(intrate).transpose()
# the independent variables shaped as columns
x1 = np.array(fico).transpose()
x2 = np.array(loanamt).transpose()

# put the two columns together to create an input matrix 
# if we had n independent variables we would have n columns here
x = np.column_stack([x1,x2])

# create a linear model and fit it to the data
X = sm.add_constant(x)
model = sm.OLS(y,X)
f = model.fit()
print('Coefficients: ', f.params[0:2])
print('Intercept: ', f.params[2])
print('P-Values: ', f.pvalues)
print('R-Squared: ', f.rsquared)

Coefficients:  [65.8744788  -0.08520339]
Intercept:  0.18193967135306596
P-Values:  [0.00000000e+000 0.00000000e+000 4.27159025e-258]
R-Squared:  0.6898129718512984


#SkLearn Linear Regression

In [159]:
x = lin_Data.drop(['Interest.Rate','Loan.Purpose','Debt.To.Income.Ratio','State','Home.Ownership','Employment.Length','Revolving.CREDIT.Balance','Amount.Funded.By.Investors'], axis = 1)

#Rename columns so I can use syntax below for finding medians and changing types
x.columns = ['Amount_Requested','Loan_Length','Monthly_Income','FICO_Range','Open_CREDIT_Lines','Inquiries_in_the_Last_6_Months']

#Fill in any NaN values with medians
x.Amount_Requested.fillna(x.Amount_Requested.median())
x.Loan_Length.fillna(x.Loan_Length.median())
x.Monthly_Income.fillna(x.Monthly_Income.median())
x.FICO_Range.fillna(x.FICO_Range.median())
x.Open_CREDIT_Lines.fillna(x.Open_CREDIT_Lines.median())
x.Inquiries_in_the_Last_6_Months.fillna(0)

#Convert them all from float64 to float32
x.Amount_Requested = np.int32(x['Amount_Requested'])
x.Loan_Length = np.int32(x['Loan_Length'])
x.Monthly_Income = np.int32(x['Monthly_Income'])
x.FICO_Range = np.int32(x['FICO_Range'])
x.Open_CREDIT_Lines = np.int32(x['Open_CREDIT_Lines'])
x.Inquiries_in_the_Last_6_Months = np.int32(x['Inquiries_in_the_Last_6_Months'])

x.head()

,Amount_Requested,Loan_Length,Monthly_Income,FICO_Range,Open_CREDIT_Lines,Inquiries_in_the_Last_6_Months
81174,20000,36,6541,737,14,2
99592,19200,36,4583,717,12,1
80059,35000,60,11500,692,14,1
15825,10000,36,3833,697,10,0
33182,12000,36,3195,697,11,0


In [160]:
reg_Mod = linear_model.LinearRegression()
y = lin_Data['Interest.Rate']


reg_Mod.fit(x,y)
reg_Mod.coef_
#Shows the coefficients you would have in the equation for our line and its intercept

array([ 1.47168645e-04,  1.35415975e-01, -3.30551907e-09, -8.84235797e-02,
       -6.34028201e-02,  6.34028216e-02])

In [161]:
reg_Mod.intercept_

68.8283971533733

In [164]:
reg_Mod.predict([[20000,36,6541,737,14,2]])
#Lets predict interest rate based on requesting 20,000 loan of 36 months with a monthly income of 6,541, a FICO score of 737, 14 open credit lines and 2 inquiries in the last 6 months

array([10.71771143])

In [165]:
reg_Mod.predict([[50000,12,6541,737,14,2]]) #Let's see how changing it to a larger loan and a shorter time period affects the interest rate

array([11.88278739])

In [167]:
reg_Mod.predict([[20000,36,6541,500,14,2]]) #What if everything from first predict was the same but I had a poor credit score?

array([31.67409982])

In [166]:
reg_Mod.score(x,y) #It's a pretty good score!

0.7499219967288923

#SkLearn Logistic Regression